# FLOODS PREDICTIONS

problems:
* (emdat) we don't have the day or even the month of some flood event (we will use the in the merge)
* (precipitation) we don't have the precipitation data until 2020 for all the countries (we will use the flood events for these countires during the merge)
* (precipitation) we don't have the precipitation for every day 
* (emdat) we don't always have the number of affected 
=> this causes a lot of loss in the data during merge 




# Packages:

In [179]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import math
import random

In [180]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, max_error
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import max_error, explained_variance_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

# Precipitation

#### Data presentation: 
We will use historical daily precipitation data that we collected from NOAA's (National Oceanic and Atmospheric Administration) National Centers for Environmental Information.

source of data: https://www.ncei.noaa.gov/data/global-historical-climatology-network-daily/

NOAA's National Centers for Environmental Information website: https://www.ncei.noaa.gov/

We created new features using the collected features in order to improve the performance of our prediction model.

The original features are: date	avg_rain,	sum_rain,	max_rain,	stddev_rain	station_count

The new features are: last_5_days_sum,	last_5_days_avg,	last_5_days_max	last_10_days_sum,	last_10_days_avg,	last_10_days_max,	last_20_days_sum,	last_20_days_avg,	last_20_days_max

Originally the data were mapped to weather stations. So, we mapped them to the coutries (using the ISO3 code) 

In [181]:
# read precipitation data
precipitation = pd.read_csv(
    "daily_rain_by_country_feature.csv.gz",
    compression="gzip",
    header=0,
    sep=",",
    quotechar='"',
    error_bad_lines=False,
)

In [182]:
pd.set_option("display.max_columns", None)
precipitation.head()

,country_ISO3,date,avg_rain,sum_rain,max_rain,stddev_rain,station_count,last_5_days_sum,last_5_days_avg,last_5_days_max,last_10_days_sum,last_10_days_avg,last_10_days_max,last_20_days_sum,last_20_days_avg,last_20_days_max
0,CCK,1949-08-19,30.0,30.0,30.0,0.0,1,60.0,30.0,30.0,60.0,30.0,30.0,60.0,30.0,30.0
1,CCK,1949-08-19,30.0,30.0,30.0,0.0,1,60.0,30.0,30.0,60.0,30.0,30.0,60.0,30.0,30.0
2,CCK,1949-09-12,89.0,89.0,89.0,0.0,1,178.0,89.0,89.0,178.0,89.0,89.0,178.0,89.0,89.0
3,CCK,1949-09-12,89.0,89.0,89.0,0.0,1,178.0,89.0,89.0,178.0,89.0,89.0,178.0,89.0,89.0
4,CCK,1950-04-12,51.0,51.0,51.0,0.0,1,102.0,51.0,51.0,102.0,51.0,51.0,102.0,51.0,51.0


In [183]:
precipitation.tail()

,country_ISO3,date,avg_rain,sum_rain,max_rain,stddev_rain,station_count,last_5_days_sum,last_5_days_avg,last_5_days_max,last_10_days_sum,last_10_days_avg,last_10_days_max,last_20_days_sum,last_20_days_avg,last_20_days_max
10225713,ATF,2018-12-20,0.0,0.0,0.0,0.0,1,122.0,10.166667,56.0,448.0,20.363636,160.0,672.0,16.000000,160.0
10225714,ATF,2018-12-21,0.0,0.0,0.0,0.0,1,112.0,9.333333,56.0,448.0,20.363636,160.0,672.0,16.000000,160.0
10225715,ATF,2018-12-21,0.0,0.0,0.0,0.0,1,112.0,9.333333,56.0,448.0,20.363636,160.0,672.0,16.000000,160.0
10225716,ATF,2018-12-22,15.0,15.0,15.0,0.0,1,30.0,2.500000,15.0,478.0,21.727273,160.0,686.0,16.333333,160.0
10225717,ATF,2018-12-22,15.0,15.0,15.0,0.0,1,30.0,2.500000,15.0,478.0,21.727273,160.0,686.0,16.333333,160.0


#### About precipitation data

In [184]:
print("the number of rows in the precipitation table:", precipitation.shape[0])
print("\nthe features in the precipitation table:\n")
print(precipitation.columns.tolist())

the number of rows in the precipitation table: 10225718

the features in the precipitation table:

['country_ISO3', 'date', ' avg_rain', 'sum_rain', 'max_rain', 'stddev_rain', 'station_count', 'last_5_days_sum', 'last_5_days_avg', 'last_5_days_max', 'last_10_days_sum', 'last_10_days_avg', 'last_10_days_max', 'last_20_days_sum', 'last_20_days_avg', 'last_20_days_max']


In [185]:
# check is we have null dates /missing dates
print("We don't have missing dates :")
precipitation[precipitation["date"].isnull()]

We don't have missing dates :


,country_ISO3,date,avg_rain,sum_rain,max_rain,stddev_rain,station_count,last_5_days_sum,last_5_days_avg,last_5_days_max,last_10_days_sum,last_10_days_avg,last_10_days_max,last_20_days_sum,last_20_days_avg,last_20_days_max


#### converting the date from string type to datetime type

In [186]:
# converting the date from string type to datetime type
precipitation["date"] = pd.to_datetime(precipitation["date"])

#### splitting  the date into day, month and year 

In [187]:
# split the date into day , month and year
precipitation["Start Day"] = precipitation.date.dt.day
precipitation["Start Month"] = precipitation.date.dt.month
precipitation["Start Year"] = precipitation.date.dt.year

In [188]:
precipitation.head()

,country_ISO3,date,avg_rain,sum_rain,max_rain,stddev_rain,station_count,last_5_days_sum,last_5_days_avg,last_5_days_max,last_10_days_sum,last_10_days_avg,last_10_days_max,last_20_days_sum,last_20_days_avg,last_20_days_max,Start Day,Start Month,Start Year
0,CCK,1949-08-19,30.0,30.0,30.0,0.0,1,60.0,30.0,30.0,60.0,30.0,30.0,60.0,30.0,30.0,19,8,1949
1,CCK,1949-08-19,30.0,30.0,30.0,0.0,1,60.0,30.0,30.0,60.0,30.0,30.0,60.0,30.0,30.0,19,8,1949
2,CCK,1949-09-12,89.0,89.0,89.0,0.0,1,178.0,89.0,89.0,178.0,89.0,89.0,178.0,89.0,89.0,12,9,1949
3,CCK,1949-09-12,89.0,89.0,89.0,0.0,1,178.0,89.0,89.0,178.0,89.0,89.0,178.0,89.0,89.0,12,9,1949
4,CCK,1950-04-12,51.0,51.0,51.0,0.0,1,102.0,51.0,51.0,102.0,51.0,51.0,102.0,51.0,51.0,12,4,1950


#### renaming columns in order to have the same column names as emdat (for columns in common)

In [189]:
# rename columns
precipitation.columns = np.array(
    [
        "ISO",
        "date",
        " avg_rain",
        "sum_rain",
        "max_rain",
        "stddev_rain",
        "station_count",
        "last_5_days_sum",
        "last_5_days_avg",
        "last_5_days_max",
        "last_10_days_sum",
        "last_10_days_avg",
        "last_10_days_max",
        "last_20_days_sum",
        "last_20_days_avg",
        "last_20_days_max",
        "Start Day",
        "Start Month",
        "Start Year",
    ]
)

#### removing duplicate rows in the precipitation table 

In [190]:
## remove duplicate date in NOAA that lead to more columns after merge with EMDAT
precipitation = precipitation[~precipitation.duplicated(keep="first")]
precipitation.shape

(5112859, 19)

# EMDAT 

#### Data presentation:

This is the table that contains the disasters (floods, storms and droughts)

For a disaster to be entered into the EM-DAT database at least one of the following criteria must be fulfilled:
(as described here:  https://www.emdat.be/explanatory-notes):
* Ten (10) or more people reported killed
* Hundred (100) or more people reported affected
* Declaration of a state of emergency
* Call for international assistance


Source: EM-DAT, CRED / UCLouvain, Brussels, Belgium ; www.emdat.be (D. Guha-Sapir)

Version: 020-09-01

File creation: Sat, 12 Sep 2020 22:21:30 CEST

Table type: Custom request

Number of records: 24520

#### loading emdat data and selecting flood events 

In [191]:
table = pd.read_excel("emdat_public_2020_09_12_query_uid-tAnKEX.xlsx", skiprows=6)
# Selecting floods
flood_table = table[table["Disaster Type"] == "Flood"]
# deleting columns with nans only
for column in flood_table.columns:
    if flood_table[column].isnull().all():
        del flood_table[column]

In [192]:
flood_table.head()

,Dis No,Year,Seq,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Event Name,Entry Criteria,Country,ISO,Region,Continent,Location,Origin,Associated Dis,Associated Dis2,OFDA Response,Appeal,Declaration,Aid Contribution,Dis Mag Value,Dis Mag Scale,Latitude,Longitude,River Basin,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,No Injured,No Affected,No Homeless,Total Affected,Reconstruction Costs ('000 US$),Insured Damages ('000 US$),Total Damages ('000 US$),CPI
16,1906-0023-BEL,1906,23,Natural,Hydrological,Flood,NaN,NaN,Govern,Belgium,BEL,Western Europe,Europe,Louvain region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Km2,NaN,NaN,NaN,1906,5.0,14.0,1906,5.0,14.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.522300
17,1906-0024-BEL,1906,24,Natural,Hydrological,Flood,NaN,NaN,Govern,Belgium,BEL,Western Europe,Europe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Km2,NaN,NaN,NaN,1906,4.0,NaN,1906,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.522300
50,1911-0002-CHN,1911,2,Natural,Hydrological,Flood,Riverine flood,NaN,Kill,China,CHN,Eastern Asia,Asia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Km2,NaN,NaN,NaN,1911,NaN,NaN,1911,NaN,NaN,100000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.652756
63,1915-0011-CHN,1915,11,Natural,Hydrological,Flood,Riverine flood,NaN,Affect,China,CHN,Eastern Asia,Asia,Guangzhou,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3000.0,Km2,NaN,NaN,NaN,1915,NaN,NaN,1915,NaN,NaN,NaN,NaN,3000000.0,NaN,3000000.0,NaN,NaN,NaN,3.965849
79,1917-0013-CHN,1917,13,Natural,Hydrological,Flood,Riverine flood,NaN,Affect,China,CHN,Eastern Asia,Asia,Tianjin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Km2,NaN,NaN,NaN,1917,NaN,NaN,1917,NaN,NaN,NaN,NaN,635000.0,NaN,635000.0,NaN,NaN,NaN,5.022539


In [193]:
print(flood_table.shape)
flood_table.columns

(5277, 41)


Index(['Dis No', 'Year', 'Seq', 'Disaster Group', 'Disaster Subgroup',
       'Disaster Type', 'Disaster Subtype', 'Event Name', 'Entry Criteria',
       'Country', 'ISO', 'Region', 'Continent', 'Location', 'Origin',
       'Associated Dis', 'Associated Dis2', 'OFDA Response', 'Appeal',
       'Declaration', 'Aid Contribution', 'Dis Mag Value', 'Dis Mag Scale',
       'Latitude', 'Longitude', 'River Basin', 'Start Year', 'Start Month',
       'Start Day', 'End Year', 'End Month', 'End Day', 'Total Deaths',
       'No Injured', 'No Affected', 'No Homeless', 'Total Affected',
       'Reconstruction Costs ('000 US$)', 'Insured Damages ('000 US$)',
       'Total Damages ('000 US$)', 'CPI'],
      dtype='object')

#### missing dates (start day, start month or start year) of floods 

These dates are important to have as we will merge the table of floods with the daily prcipitation table.
If we don't have the exact date we won't have the exact precipitation data. 

In [194]:
print("number of events with unknown start year")
print(flood_table[flood_table["Start Year"].isnull()].shape[0])
print("number of events with unknown start month")
print(flood_table[flood_table["Start Month"].isnull()].shape[0])
print("number of events with unknown start day")
print(flood_table[flood_table["Start Day"].isnull()].shape[0])
print("number of events with unknown end Year")
print(flood_table[flood_table["End Year"].isnull()].shape[0])
print("number of events with unknown end month")
print(flood_table[flood_table["End Month"].isnull()].shape[0])
print("number of events with unknown end day")
print(flood_table[flood_table["End Day"].isnull()].shape[0])
### 929 events don't have the exact date and can't have the precipitation information
print("number of events with unknown start month but known end month")
print(
    flood_table[
        flood_table["Start Month"].isnull() & flood_table["End Month"].notnull()
    ].shape[0]
)
print("number of events with unknown start day but known end day")
print(
    flood_table[
        flood_table["Start Day"].isnull() & flood_table["End Day"].notnull()
    ].shape[0]
)

number of events with unknown start year
0
number of events with unknown start month
51
number of events with unknown start day
929
number of events with unknown end Year
0
number of events with unknown end month
50
number of events with unknown end day
941
number of events with unknown start month but known end month
1
number of events with unknown start day but known end day
17


#### handling missing dates  (start day or start month)  when we have  end day or end month
 if we have the end day (end month) and we don't have the start day (start month), we use the values of end day (end month) instead. 


In [195]:
## if we have the end day (end month) and we don't have the start day (start month), we use the values of end day (end month) instead.
flood_table["Start Day"] = flood_table["Start Day"].fillna(flood_table["End Day"])
flood_table["Start Month"] = flood_table["Start Month"].fillna(flood_table["End Month"])
print(flood_table[flood_table["Start Day"].isnull()].shape[0])
print(flood_table[flood_table["Start Month"].isnull()].shape[0])

912
50


C:\Users\GAMDOU\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\GAMDOU\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


#### handling missing values when we don't have end day or end month 
to handle missing start days/ start months when we don't know the end days/ end months:

* we can choose 15 for the day in order to assure that the error in each date is no more than ±16
* we can choose 6 for the month in order to assure that the error in each date is no more than ±7

However, according to https://journals.sagepub.com/doi/pdf/10.1177/1536867X1201200110 this is called mean imputation and it is undesirable because it produces spikes at the mean and distorts distributional parameters
such as the variance

So, another solution is to select random values for these missing values 

In [196]:
flood_table["Start Day"] = flood_table["Start Day"].fillna(random.randint(1, 28))
## replacing missing months by the 6th of the month
flood_table["Start Month"] = flood_table["Start Month"].fillna(random.randint(1, 12))
print(flood_table[flood_table["Start Day"].isnull()].shape[0])
print(flood_table[flood_table["Start Month"].isnull()].shape[0])

0
0


C:\Users\GAMDOU\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\GAMDOU\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


# Predicting the number of total affected 


# Model for prediction
merging the precipitation table with the flood table (from emdat) in order to have the precipitation data for each flood event

However, we will lose some rows as we don't have some precipitation data for some country in  some given dates (the precipitation table has some missing data). In fact, as our prediction is based on precipitation data, we will not use the events that don't have precipitation data in our prediction.

In [197]:
# we can use left join if we can replace the nans in rain data by a mean ...
# we can use the full flood table (even events without a full date) if we find a solution to nans in rain data
total_data = flood_table.merge(
    precipitation,
    how="inner",
    left_on=["ISO", "Start Year", "Start Month", "Start Day"],
    right_on=["ISO", "Start Year", "Start Month", "Start Day"],
)
print(
    "the number of rows :",
    total_data.shape[0],
    "\nthe number of features :",
    total_data.shape[1],
)
print("\nthe features :", total_data.columns.tolist())

the number of rows : 4027 
the number of features : 56

the features : ['Dis No', 'Year', 'Seq', 'Disaster Group', 'Disaster Subgroup', 'Disaster Type', 'Disaster Subtype', 'Event Name', 'Entry Criteria', 'Country', 'ISO', 'Region', 'Continent', 'Location', 'Origin', 'Associated Dis', 'Associated Dis2', 'OFDA Response', 'Appeal', 'Declaration', 'Aid Contribution', 'Dis Mag Value', 'Dis Mag Scale', 'Latitude', 'Longitude', 'River Basin', 'Start Year', 'Start Month', 'Start Day', 'End Year', 'End Month', 'End Day', 'Total Deaths', 'No Injured', 'No Affected', 'No Homeless', 'Total Affected', "Reconstruction Costs ('000 US$)", "Insured Damages ('000 US$)", "Total Damages ('000 US$)", 'CPI', 'date', ' avg_rain', 'sum_rain', 'max_rain', 'stddev_rain', 'station_count', 'last_5_days_sum', 'last_5_days_avg', 'last_5_days_max', 'last_10_days_sum', 'last_10_days_avg', 'last_10_days_max', 'last_20_days_sum', 'last_20_days_avg', 'last_20_days_max']


#### handling missing data in total affected column (our target column) :
We use the mean number of total affected in a country to replace missing data in the column of total affected (during a flood event in a given country)

If we don't have the mean number of total affected in a country for a given year we take the mean of total affected in the region for this given year. If we don't have both values we take the mean of total affected over all the period (all the years considered by emdat)

In [198]:
## handling missing data in total affected column :
affected_by_country_per_year = (
    total_data[["Region", "ISO", "Total Affected", "Start Year"]]
    .groupby(by=["Region", "ISO", "Start Year"])
    .mean()
)

affected_by_region_per_year = (
    total_data[["Region", "Total Affected", "Start Year"]]
    .groupby(by=["Region", "Start Year"])
    .mean()
)
affected_by_region_total = (
    total_data[["Region", "Total Affected"]].groupby(by=["Region"]).mean()
)
# if we don't have the mean number of total affected in a country for a given year
# we take the mean of the region for the given year, and if we don't have both we take the mean over all the period
for i in affected_by_region_per_year.index:
    if math.isnan(affected_by_region_per_year.loc[i, "Total Affected"]):
        affected_by_region_per_year.loc[
            i, "Total Affected"
        ] = affected_by_region_total.loc[i[0], "Total Affected"]

for i in affected_by_country_per_year.index:
    if math.isnan(affected_by_country_per_year.loc[i, "Total Affected"]):
        affected_by_country_per_year.loc[
            i, "Total Affected"
        ] = affected_by_region_per_year.loc[(i[0], i[2]), "Total Affected"]
affected_by_country_per_year

Total Affected
Region                    ISO Start Year                
Australia and New Zealand AUS 1955           4495.075000
                              1971           4495.075000
                              1973          12000.000000
                              1974           4495.075000
                              1975           4495.075000
...                                                  ...
Western Europe            LUX 1993          37866.666667
                          NLD 1953         300000.000000
                              1992          15299.367647
                              1993          13000.000000
                              1998           2000.000000

[2164 rows x 1 columns]

In [199]:
## handling missing data in total affected column :
for i in total_data.index:
    if math.isnan(total_data.loc[i, "Total Affected"]):
        total_data.loc[i, "Total Affected"] = affected_by_country_per_year.loc[
            (
                total_data.loc[i, "Region"],
                total_data.loc[i, "ISO"],
                total_data.loc[i, "Start Year"],
            ),
            "Total Affected",
        ]

#### selecting features

choosing the features for X and the target y:


In [200]:
#'Start Month','Start Day',, 'Continent','last_5_days_sum',
X = total_data[
    [
        "ISO",
        "Region",
        "Start Year",
        "Entry Criteria",
        " avg_rain",
        "sum_rain",
        "max_rain",
        "stddev_rain",
        "last_5_days_avg",
        "last_5_days_max",
        "last_10_days_avg",
        "last_10_days_max",
        "last_20_days_avg",
        "last_20_days_max",
    ]
]


y = total_data["Total Affected"]

In [201]:
# our target
y

0       1.529937e+04
1       1.529937e+04
2       1.529937e+04
3       2.589375e+06
4       7.416576e+04
            ...     
4022    6.000000e+04
4023    3.900000e+02
4024    1.800000e+04
4025    4.335000e+04
4026    9.258000e+04
Name: Total Affected, Length: 4027, dtype: float64

#### data preparation 

cleaning Entry Criteria data

In [202]:
# cleaning Entry Criteria

print(X["Entry Criteria"].unique())
X["Entry Criteria"] = X["Entry Criteria"].fillna("Unknown")
X["Entry Criteria"] = X["Entry Criteria"].apply(lambda x: "Unknown" if x == "--" else x)
X["Entry Criteria"].unique()

['Govern' 'OFDA' 'Kill' 'Affect' 'Declar' 'SigDam' 'Affected' 'Unknown'
 'Regional' 'Declar/Int' 'Waiting' 'SigDis' nan '--']


C:\Users\GAMDOU\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\GAMDOU\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


array(['Govern', 'OFDA', 'Kill', 'Affect', 'Declar', 'SigDam', 'Affected',
       'Unknown', 'Regional', 'Declar/Int', 'Waiting', 'SigDis'],
      dtype=object)

#### replacing non numeric data by numeric values 

In [203]:
# replacing ISO by numeric values
ISO_dic = {}
count = 0
for iso in X["ISO"].unique():
    ISO_dic[iso] = count
    count += 1
X["ISO"] = X["ISO"].apply(lambda x: ISO_dic[x])
# replacing Region by numeric values
Region_dic = {}
count = 0
for region in X["Region"].unique():
    Region_dic[region] = count
    count += 1
X["Region"] = X["Region"].apply(lambda x: Region_dic[x])
# replacing Continent by numeric values
# Continent_dic={}
# count=0
# for continent in X['Continent'].unique():
#    Continent_dic[continent]=count
#    count+=1
# X['Continent']=X['Continent'].apply(lambda x: Continent_dic[x])
# replacing Entry Criteria by numeric values
Entry_Criteria_dic = {}
count = 0

for criteria in X["Entry Criteria"].unique():
    if criteria == "Unknown":
        Entry_Criteria_dic[criteria] = -1
    else:
        Entry_Criteria_dic[criteria] = count
        count += 1
X["Entry Criteria"] = X["Entry Criteria"].apply(lambda x: Entry_Criteria_dic[x])

C:\Users\GAMDOU\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\GAMDOU\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\GAMDOU\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

In [204]:
X.head()

,ISO,Region,Start Year,Entry Criteria,avg_rain,sum_rain,max_rain,stddev_rain,last_5_days_avg,last_5_days_max,last_10_days_avg,last_10_days_max,last_20_days_avg,last_20_days_max
0,0,0,1906,0,13.00,13.0,13.0,0.00,39.833333,117.0,22.181818,117.0,26.047619,117.0
1,0,0,1906,0,0.00,0.0,0.0,0.00,0.000000,0.0,0.363636,4.0,0.761905,5.0
2,0,0,1926,0,0.00,0.0,0.0,0.00,77.000000,282.0,46.000000,282.0,35.333333,282.0
3,1,1,1926,1,132.19,277585.0,3353.0,259.28,113.951667,4854.0,106.884545,4854.0,107.260476,4854.0
4,2,2,1927,2,48.87,733.0,230.0,70.69,31.686667,250.0,33.875455,264.0,38.142381,390.0


#### normalizing rain data  using statistical normalization 

In [205]:
# normalizing rain data  using statistical normalization

list_rain_features = [
    " avg_rain",
    "sum_rain",
    "max_rain",
    "stddev_rain",
    "last_5_days_avg",
    "last_5_days_max",
    "last_10_days_avg",
    "last_10_days_max",
    "last_20_days_avg",
    "last_20_days_max",
]

mean_normalization = {}
std_normalization = {}
for feature in list_rain_features:
    mean_feature = X[feature].mean()
    mean_normalization[feature] = mean_feature
    std_feature = X[feature].std()
    std_normalization[feature] = std_feature
    X[feature] = (X[feature] - mean_feature) / std_feature

C:\Users\GAMDOU\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [206]:
X.head()

,ISO,Region,Start Year,Entry Criteria,avg_rain,sum_rain,max_rain,stddev_rain,last_5_days_avg,last_5_days_max,last_10_days_avg,last_10_days_max,last_20_days_avg,last_20_days_max
0,0,0,1906,0,-0.492732,-0.266158,-0.771022,-0.824077,-0.430867,-0.743318,-0.723585,-0.729324,-0.698325,-0.694531
1,0,0,1906,0,-0.571805,-0.266250,-0.786352,-0.824077,-0.917825,-0.829078,-1.054237,-0.796021,-1.147369,-0.745842
2,0,0,1926,0,-0.571805,-0.266250,-0.786352,-0.824077,0.023490,-0.622373,-0.362623,-0.631936,-0.533422,-0.618939
3,1,1,1926,1,0.232247,1.691093,3.167642,0.886692,0.475220,2.728892,0.560076,2.066613,0.743918,1.475644
4,2,2,1927,2,-0.274551,-0.261081,-0.515127,-0.357653,-0.530459,-0.645829,-0.546369,-0.642560,-0.483537,-0.569461


#### splitting data train/test: 

In [207]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=123
)
X_train, X_validation, y_train, y_validation = train_test_split(
    X_train, y_train, test_size=0.1, random_state=123
)
print("X_train ", X_train.shape)
print("y_train ", X_train.shape)
print("X_test", X_test.shape)
print("y_test", X_test.shape)
print("X_validation", X_validation.shape)
print("y_validation", X_validation.shape)

X_train  (2898, 14)
y_train  (2898, 14)
X_test (806, 14)
y_test (806, 14)
X_validation (323, 14)
y_validation (323, 14)


## GradientBoostingRegressor

#### selection of the model parameters using the validation set 

In [208]:
best_regr_score = 0
for learning_rate in [0.01, 0.03, 0.05, 0.1, 0.3, 0.5]:
    for n_estimators in [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 100, 110]:
        regr1 = GradientBoostingRegressor(
            random_state=0, learning_rate=learning_rate, n_estimators=n_estimators
        )
        regr1.fit(X_train, y_train)
        y_pred = regr1.predict(X_validation)
        regr1_score = regr1.score(X_validation, y_validation)
        if regr1_score > best_regr_score:
            best_regr_score = regr1_score
            best_learning_rate = learning_rate
            best_n_estimators = n_estimators

print("best_regr_score", best_regr_score)
print("best_learning_rate", best_learning_rate)
print("best_n_estimators", best_n_estimators)

best_regr_score 0.2886662811827799
best_learning_rate 0.05
best_n_estimators 100


#### model set-up and prediction 

In [209]:
regr1 = GradientBoostingRegressor(
    random_state=0, learning_rate=best_learning_rate, n_estimators=best_n_estimators
)
regr1.fit(X_train, y_train)
y_pred1 = regr1.predict(X_test)

#### prediction metrics (per country per event)

In [210]:
print("regr.score: The best possible score is 1.0")
print(regr1.score(X_test, y_test))
print("mean_absolute_error")
print(mean_absolute_error(y_test, y_pred1))
print("max_error")
print(max_error(y_test, y_pred1))
print("explained_variance_score: The best possible score is 1.0")
print(explained_variance_score(y_test, y_pred1))

regr.score: The best possible score is 1.0
0.10485451403798185
mean_absolute_error
1864281.2405204533
max_error
200952516.21164772
explained_variance_score: The best possible score is 1.0
0.10533213298832367


#### prediction metrics (per region per year)

In [211]:
Y_agg = pd.DataFrame()
Y_agg["Total Affected"] = y_test
Y_agg["predicted total Affected"] = y_pred1
Y_agg["prediction error"] = y_test - y_pred1
Y_agg["ISO"] = X["ISO"]
Y_agg["Region"] = X["Region"]
Y_agg["Start Year"] = X["Start Year"]
Y_agg["Entry Criteria"] = X["Entry Criteria"]

Y_agg

,Total Affected,predicted total Affected,prediction error,ISO,Region,Start Year,Entry Criteria
1121,29000.0,-3.418310e+05,3.708310e+05,5,5,1997,3
3626,3000.0,3.943533e+04,-3.643533e+04,137,15,2018,6
2167,126096.0,9.675925e+04,2.933675e+04,22,3,2006,2
1179,18000.0,1.662987e+06,-1.644987e+06,56,16,1994,6
198,3000000.0,4.482978e+06,-1.482978e+06,31,4,1965,2
...,...,...,...,...,...,...,...
3010,89000.0,-3.120585e+05,4.010585e+05,3,3,2012,6
803,19500.0,3.556333e+06,-3.536833e+06,3,3,1992,2
1304,12000.0,5.924713e+05,-5.804713e+05,29,13,1998,2
3853,1800.0,4.877582e+05,-4.859582e+05,88,1,2019,6


In [212]:
Y_final = (
    Y_agg[["Region", "Start Year", "predicted total Affected", "Total Affected"]]
    .groupby(by=["Region", "Start Year"])
    .sum()
)
Y_final["absolute difference"] = (
    Y_final["predicted total Affected"] - Y_final["Total Affected"]
).apply(lambda x: abs(x))
print("mean absolute error")
Y_final["absolute difference"].mean()

mean absolute error


3150508.754581551

## RandomForestRegressor

#### selection of the model parameters using the validation set 

In [213]:
best_regr_score = 0
for n_estimators in [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 100, 110]:
    for min_samples_split in range(2, 11):
        regr = RandomForestRegressor(
            n_estimators=n_estimators,
            min_samples_split=min_samples_split,
            random_state=2,
        )
        regr.fit(X_train, y_train)
        y_pred = regr.predict(X_validation)
        regr_score = regr.score(X_validation, y_validation)
        if regr_score > best_regr_score:
            best_regr_score = regr_score
            best_n_estimators = n_estimators
            best_min_samples_split = min_samples_split

print("best_regr_score", best_regr_score)
print("best_n_estimators", best_n_estimators)
print("best_min_samples_split", best_min_samples_split)

best_regr_score 0.13414078495643833
best_n_estimators 100
best_min_samples_split 9


#### model set-up and prediction 

In [214]:
regr = RandomForestRegressor(
    n_estimators=best_n_estimators,
    min_samples_split=best_min_samples_split,
    random_state=2,
)
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
print(regr.score(X_test, y_test))

0.0987942769590543


#### prediction metrics (per country per event)

In [215]:
# he coefficient of determination R^2 of the prediction
print("regr.score: The best possible score is 1.0")
print(regr.score(X_test, y_test))
print("mean_absolute_error")
print(mean_absolute_error(y_test, y_pred))
print("max_error")
print(max_error(y_test, y_pred))
print("explained_variance_score: The best possible score is 1.0")
print(explained_variance_score(y_test, y_pred))

regr.score: The best possible score is 1.0
0.0987942769590543
mean_absolute_error
1918355.4487497786
max_error
196346052.04449385
explained_variance_score: The best possible score is 1.0
0.09879697177625313


#### prediction metrics (per region per year)

In [216]:
Y_agg = pd.DataFrame()
Y_agg["Total Affected"] = y_test  # *std_y+mean_y
Y_agg["predicted total Affected"] = y_pred  # *std_y+mean_y
Y_agg["prediction error"] = y_test - y_pred  # *std_y+mean_y
Y_agg["ISO"] = X["ISO"]
Y_agg["Region"] = X["Region"]
# Y_agg['Continent']=X['Continent']
Y_agg["Start Year"] = X["Start Year"]
Y_agg["Entry Criteria"] = X["Entry Criteria"]
# Y_agg['Start Month']=X['Start Month']
# Y_agg['Start Day']=X['Start Day']
Y_agg

,Total Affected,predicted total Affected,prediction error,ISO,Region,Start Year,Entry Criteria
1121,29000.0,3.341092e+04,-4.410922e+03,5,5,1997,3
3626,3000.0,4.762564e+04,-4.462564e+04,137,15,2018,6
2167,126096.0,3.039275e+05,-1.778315e+05,22,3,2006,2
1179,18000.0,4.143685e+05,-3.963685e+05,56,16,1994,6
198,3000000.0,1.533868e+06,1.466132e+06,31,4,1965,2
...,...,...,...,...,...,...,...
3010,89000.0,8.502440e+05,-7.612440e+05,3,3,2012,6
803,19500.0,1.426843e+06,-1.407343e+06,3,3,1992,2
1304,12000.0,3.242022e+04,-2.042022e+04,29,13,1998,2
3853,1800.0,5.713252e+05,-5.695252e+05,88,1,2019,6


In [217]:
Y_final = (
    Y_agg[["Region", "Start Year", "predicted total Affected", "Total Affected"]]
    .groupby(by=["Region", "Start Year"])
    .sum()
)
Y_final["absolute difference"] = (
    Y_final["predicted total Affected"] - Y_final["Total Affected"]
).apply(lambda x: abs(x))
print("mean absolute error")
Y_final["absolute difference"].mean()

mean absolute error


3174448.9507525126